## Libraries und Vairablen

In [29]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by  import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time
import re

In [30]:
buchhaus_new_last_30 = 'https://www.buchhaus.ch/de/heute/last30'

## Webscraping

In [31]:
PATH = "C:\Program Files (x86)\chromedriver.exe"
driver = webdriver.Chrome(PATH)
#load the page
driver.get(buchhaus_new_last_30)

# data frame
buch_df = pd.DataFrame(columns=['title',  
                            'preis',
                            'untertitel',
                            'autor', 
                            'details', 
                            'genre',
                            'text'])

''' 
Algorith explanation:
1. go to the i'th book and click on it
2. extract all the necessary data
3. add the data to the data frame
4. go back to main page
5. after 25 books go to the next page
'''

for page in range(1, 10):
    for book in range(1,8):
        try:
            time.sleep(1)
            # go to the i'th book and click on it
            buch = driver.find_element(By.XPATH, '/html/body/div[2]/div/div/main/div[3]/div[2]/div[4]/div/div/div[{}]/div/div[1]/div/div/span/span/a'.format(book))
            buch.click()

            # get the data
            try: titel = driver.find_element(By.XPATH, '/html/body/div[2]/div/div/main/div[2]/section[1]/div[2]/div[1]/h1/span/span').text
            except: titel = 'nan'

            try: preis = driver.find_element(By.XPATH, '/html/body/div[2]/div/div/main/div[2]/section[1]/div[2]/div[2]/div/div/span[2]/span').text
            except: preis = 'nan'

            try: untertitel = driver.find_element(By.XPATH, '/html/body/div[2]/div/div/main/div[2]/section[1]/div[2]/div[1]/div[2]/div').text
            except: untertitel = 'nan'

            try: autor = driver.find_element(By.XPATH, '/html/body/div[2]/div/div/main/div[2]/section[1]/div[2]/div[1]/div[2]/div/span/span/span/span/a').text
            except: autor = 'nan'

            # get the 'Buchbeschreibungen'
            try: text = driver.find_element(By.XPATH, '/html/body/div[2]/div/div/main/div[2]/section[3]/div[1]/div[2]/div/span/span/span').text
            except: text = 'nan'

            # get the 'Buchdetails'
            try: details = driver.find_element(By.XPATH, '/html/body/div[2]/div/div/main/div[2]/section[3]/div[2]/div[2]/div').text
            except: details = 'nan'

        # Fehlermeldung
        except: 
            print('error mit Buch {} auf Seite {}'.format(book, page))
            continue
    

        df_temp = pd.DataFrame({'title': titel,  
                        'preis': preis,
                        'untertitel': untertitel,
                        'autor': autor,
                        'details': details,
                        'text': text
                        }, index=[0])
        buch_df = pd.concat([buch_df, df_temp], ignore_index=True)
    
        #back to the main page
        driver.get(buchhaus_new_last_30)

    # go to the next page
    if page == 1: driver.find_element(By.XPATH, '/html/body/div[2]/div/div/main/div[3]/div[2]/div[1]/div[5]/div/a').click()
    else: driver.find_element(By.XPATH, '/html/body/div[2]/div/div/main/div[3]/div[2]/div[1]/div[5]/div/a[2]').click()
    time.sleep(1)


C:\Users\yanni\AppData\Local\Temp\ipykernel_15848\2562079825.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(PATH)


error mit Buch 5 auf Seite 1
error mit Buch 6 auf Seite 1
error mit Buch 7 auf Seite 1
error mit Buch 5 auf Seite 2
error mit Buch 6 auf Seite 2
error mit Buch 7 auf Seite 2
error mit Buch 5 auf Seite 3
error mit Buch 6 auf Seite 3
error mit Buch 7 auf Seite 3
error mit Buch 5 auf Seite 4
error mit Buch 6 auf Seite 4
error mit Buch 7 auf Seite 4
error mit Buch 5 auf Seite 5
error mit Buch 6 auf Seite 5
error mit Buch 7 auf Seite 5
error mit Buch 5 auf Seite 6
error mit Buch 6 auf Seite 6
error mit Buch 7 auf Seite 6
error mit Buch 5 auf Seite 7
error mit Buch 6 auf Seite 7
error mit Buch 7 auf Seite 7
error mit Buch 5 auf Seite 8
error mit Buch 6 auf Seite 8
error mit Buch 7 auf Seite 8
error mit Buch 5 auf Seite 9
error mit Buch 6 auf Seite 9
error mit Buch 7 auf Seite 9


## Datawrangling

In [32]:
buch_df.shape

(36, 7)

In [38]:
buch_df['details'][0]

'ISBN/GTIN978-3-7645-0771-8ProduktartBuchEinbandartPaperbackVerlagBlanvaletErscheinungsdatum26.04.2023Auflage1. A.ReiheMilla Nova ermitteltReihen-Nr.05Seiten608 SeitenSpracheDeutsch'

In [40]:
data = buch_df['details'][2]

# Liste der Schlüssel, die in den Daten gefunden werden können
keys = ["ISBN/GTIN", "Produktart", "Einbandart", "Verlag", "Erscheinungsdatum", "Auflage", "Reihe", "Reihen-Nr.", "Seiten", "Sprache", "Masse", "Artikel-Nr."]

# Erstellt einen regulären Ausdruck, der nach den Schlüsseln sucht, gefolgt von beliebigen Zeichen bis zum nächsten Schlüssel
regex = "(" + "|".join(keys) + ")(.*?)(?=" + "|".join(keys) + "|$)"

# Findet alle Übereinstimmungen in den Daten
matches = re.findall(regex, data)

# Erstellt einen Datenrahmen aus den gefundenen Übereinstimmungen
df = pd.DataFrame(matches, columns=["Key", "Value"])

# Entfernt Leerzeichen am Anfang und Ende der Werte
#df["Value"] = df["Value"].str.strip()

print(df)

                 Key                                              Value
0          ISBN/GTIN                                  978-3-406-79993-8
1         Produktart                                               Buch
2         Einbandart                                           Gebunden
3             Verlag                                               Beck
4  Erscheinungsdatum                                         24.04.2023
5             Seiten                                               265 
6             Seiten                                                   
7            Sprache                                            Deutsch
8              Masse  Breite 139 mm, Höhe 217 mm, Dicke 25 mmGewicht...
9        Artikel-Nr.                                           55652787


## Ausblick

nächste Schritte:

* anpassung der Unterachiedlichen Buchinformationen

Ausblick:

* Bücher in den nächsten 30 Tagen könnten auch gescrapet werden